## The Bilayer Builder: Creating Customizable Initial Configurations of Lipid Bilayers using mBuild

Welcome to the Bilayer Builder tutorial! In this lesson, you will learn how to use the aforementioned tool to create custom bilayer systems for molecular dynamics simulation.

**Note**: This is a tutorial meant to explain how to use a computational tool. It is not a lesson on lipid bilayers as a whole, and it assumes basic knowledge of these types of systems (terms like area per lipid and headgroup offset will be used assuming the user knows what they mean).

#### Lesson 1: Basic Syntax

Let's start off with some import statements to bring in mBuild, the Bilayer class itself, as well as the lipids we'll put into our bilayer.

In [4]:
import mbuild as mb
from mbuild.recipes.bilayer.bilayer import Bilayer
#from mbuild.lib.prototypes import DSPC

Great, now let's just create a simple bilayer, and then start digging into how this tool works.

In [5]:
bi = Bilayer(lipids=[(DSPC(), 1.0, 0, 0)])

bi.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1241: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the UFF force field
  "Open Babel and the {} force field".format(forcefield))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2108: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2110: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound

Okay, a couple of things to note immediately:

* Time: the bilayer builder has to make a lot of calculations in order to do its job, so making these systems takes a while. However, this is merely a tutorial, so in the future we'll use molecules with alkyl hydrogens removed, and we'll simulate smaller systems
* The `lipids` parameter: that is the single required parameter (for now), and will be the subject of our next discussion

More importantly, let's briefly discuss how these bilayers are constructed:
* Lipid placement
    * The Bilayer Builder utilizes mBuild's `2DGridPattern` class, which creates a scaffold of positions that the lipids are placed onto. The size of this grid is controlled by the user.
* Solvation
    * Lipid bilayers do not exist in a vacuum; therefore, the Bilayer Builder has built-in capability to solvate the bilayer in water, which the user can control

The Bilayer Builder's strength is the control it gives to the user in determining physical properties of their specific lipid bilayer system. Therefore, there are numerous parameters which the Bilayer Builder requires (most are set to intelligent default values, and you would do good to minimize altering those defaults; however, this tutorial will explain how to control those variables). Let's begin with how to specify your desired composition.

#### Lesson 2: Controlling the Composition

*Parameters*:
1. `lipids`: This parameter specifies the composition of your bilayer, as well as a number of other lipid-specific parameters. This is the most involved parameter, and therefore merits more discussion
    * Data Type: `list of tuples`, where each tuple represents a single lipid molecule
    * Each tuple has four elements that must be given in this order:
        * The lipid itself; an `mb.Compound`
        * The fraction of the bilayer made up of that lipid; type `float`, must be between 0 and 1
        * Headgroup Offset: some lipid head groups tend to sink deeper into the bilayer when simulated alongside a phospholipid, so the Bilayer Builder tries to accomodate this by adjusting their starting position with respect to the bilayer normal with this variable
            * Data Type: float; greater than zero moves the lipid "up" (into the solvent), less than zero move the lipid deeper into the bilayer
        * Reference atom: Each lipid is an mBuild `Compound`, and therefore is made up of `Particles`, all of which have an integer index. The reference atom is the atom which the user considers to be the top of the "head group" of the lipid.
            * This atom will form the bilayer-solvent interface; it is therefore necessary that the user know the components of any lipid they input into the Bilayer Builder
            
*Examples*:

A `lipids` parameter for a pure DSPC bilayer:
```
lipids = [(DSPC, 1.0, 0, 0)]
```

Create a bilayer composed of 50% DSPC and 50% C-16 alcohol (remember, we want to use united-atom molecules from now on to save time, and want to keep our bilayers to a reasonable size, so 2x2 should be fine).
**Help**: Use the list provided below for your `lipids` parameter. We've gotten you started with your imports and Bilayer instantiation, just add the lipids parameter.
```
lipids = [(DPSCUA(), 0.5, 0, 0), (ALCUA(16), 0.5, -0.4, 17)] 
```


In [9]:
from mbuild.lib.UA_molecules import *

bilayer = Bilayer(lipids = [(DSPCUA(), 0.5, 0, 0), (ALCUA(16), 0.5, -0.4, 17)], n_lipids_x=2, n_lipids_y=2, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The install

Hopefully you observe a couple of things:
* There is no solvent because that is what was specified by the `solvent_per_lipid` parameter
* There are two of each kind of lipid in each leaflet, because we specified a 2x2 bilayer with the following new parameters
    2. `n_lipids_x`: The number of lipids in the x-dimension. Must be an integer. Controls the number of grid points along with n_lipids_y
    3. `n_lipids_y`: The number of lipids in the y-dimension. Must be an integer.
    
This worked out nicely because we specified fractions that made sense for the size we gave. As you use this tool, be sure to watch out for the following mistakes:

In [10]:
lipids = [(DSPCUA(), 0.33, 0, 0), (ALCUA(16), 0.33, -0.4, 17), (FFAUA(16, ester=False), 0.33, -0.4, 17)]
bilayer = Bilayer(lipids, n_lipids_x=2, n_lipids_y=2, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)


ValueError: Lipid fractions do not add up to 1.

Whoops! If you are going to specify more than one component, make sure that all of the fractions add up to 1.0; otherwise, your composition is non-sensical.

Let's fix that problem by changing one of the 0.33 to 0.34.

In [11]:
lipids = [(DSPCUA(), 0.33, 0, 0), (ALCUA(16), 0.33, -0.4, 17), (FFAUA(16, ester=False), 0.34, -0.4, 17)]
bilayer = Bilayer(lipids, n_lipids_x=2, n_lipids_y=2, solvent_per_lipid=0)
bilayer.visualize()

/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate_around_y is deprecated. Please use rotate(compound, theta, around=np.asarray([0, 1, 0]))
  warn(printed_message, DeprecationWarning)
/home/loganguy/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: rotate is deprecated. Please use Compound.rotate()
  warn(printed_message, DeprecationWarning)
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The install

The Bilayer Builder was able to create a system for you, but look closely. Each leaflet has one DSPC molecule, one alcohol molecule, and **two** free fatty acid molecules.

Presumably, this is not what we would want when we specify fractions of 0.33, 0.33, and 0.34. However, we specified a size of 2x2, and the Bilayer Builder in its current state does not tell you when the actual fraction deviates from your expected fraction.

**IMPORTANT**: Always take care that your combination of size and composition specifications make sense together.

#### Lesson 3: Controlling the individual lipid geometry